# Qcodes example with InstrumentGroup driver

This notebooks explains how to use the `InstrumentGroup` driver.

## About
The goal of the `InstrumentGroup` driver is to combine several instruments as submodules into one instrument. Typically, this is meant to be used with the `DelegateInstrument` driver. An example usage of this is to create an abstraction for devices on a chip.

## Usage
The way it's used is mainly by specifying an entry in the station YAML. For instance, to create a Chip that has one or more Devices on it that point to different source parameters. The example below shows three devices, each of which is initialised in one of the supported ways. Device1 has only DelegateParameters, while device2 and device3 have both DelegateParameters and channels added. Device3 adds its channels using a custom channel wrapper class. 

In [1]:
%%writefile example.yaml

instruments:
  dac:
    type: qcodes.instrument_drivers.mock_instruments.MockDAC
    init:
      num_channels: 3

  lockin1:
    type: qcodes.instrument_drivers.mock_instruments.MockLockin

  lockin2:
    type: qcodes.instrument_drivers.mock_instruments.MockLockin

  MockChip_123:
    type: qcodes.instrument.delegate.InstrumentGroup
    init:
      submodules_type: qcodes.instrument.delegate.DelegateInstrument
      submodules:
        device1:
          parameters:
            gate:
              - dac.ch01.voltage
            source:
              - lockin1.frequency
              - lockin1.amplitude
              - lockin1.phase
              - lockin1.time_constant
            drain:
              - lockin1.X
              - lockin1.Y
        device2:
          parameters:
            readout:
              - lockin1.phase
          channels:
            gate_1: dac.ch01
        device3:
          parameters:
            readout:
              - lockin1.phase
          channels:
            type: qcodes.instrument_drivers.mock_instruments.MockCustomChannel
            gate_1:
              channel: dac.ch02
              current_valid_range: [-0.5, 0]
            gate_2:
              channel: dac.ch03
              current_valid_range: [-1, 0]

      set_initial_values_on_load: true
      initial_values:
        device1:
          gate.step: 5e-4
          gate.inter_delay: 12.5e-4
        device2:
          gate_1.voltage.post_delay: 0.01
        device3:
          gate_2.voltage.post_delay: 0.03

Overwriting example.yaml


In [2]:
from qcodes.station import Station

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/runner/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/runner/.qcodes/logs/240226-8625-qcodes.log


In [3]:
station = Station(config_file="example.yaml")
lockin1 = station.load_lockin1()
lockin2 = station.load_lockin2()
dac = station.load_dac()
chip = station.load_MockChip_123(station=station)

In [4]:
chip.device1.gate()

0.0

In [5]:
dac.ch01.voltage()

0.0

In [6]:
chip.device1.gate(1.0)
chip.device1.gate()

1.0

In [7]:
dac.ch01.voltage()

1.0

In [8]:
chip.device1.source()

source(frequency=125.0, amplitude=0.0, phase=0.0, time_constant=0.001)

In [9]:
chip.device1.drain()

drain(X=0.001, Y=1e-05)

Device with channels/gates:

In [10]:
chip.device2.gate_1

<MockDACChannel: dac_ch01 of MockDAC: dac>

Setting voltages to a channel/gate of device2:

In [11]:
print(chip.device2.gate_1.voltage())
chip.device2.gate_1.voltage(-0.74)
print(chip.device2.gate_1.voltage())

1.0
-0.74


Check initial values of device3, from which only gate_2.voltage.post_delay was set.

In [12]:
chip.device3.gate_1.voltage.post_delay

0

In [13]:
chip.device3.gate_2.voltage.post_delay

0.03